# Deep Dive into LangChain


In [ ]:
cd /content/drive/MyDrive/Colab Notebooks/LangChain Deep Dive

/content/drive/MyDrive/Colab Notebooks/LangChain Deep Dive


In [ ]:
ls

LangChain_Deep_Dive.ipynb  requirements.txt


In [7]:
pip install -r requirements.txt -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.3/302.3 kB 12.8 MB/s eta 0:00:00


In [ ]:
#pip show openai

### Python-dotenv

In [ ]:
import os
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)

# os.environ.get('OPENAI_API_KEY')

True

# Chat Models: GPT-3.5 Turbo and GPT-4

In [10]:
pip install -q langchain_openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 437.6/437.6 kB 11.7 MB/s eta 0:00:00


In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI()
output = llm.invoke('Explain quatum mechanics in one sentence.', model='gpt-3.5-turbo')
print(output.content)

Quantum mechanics is the branch of physics that describes the behavior of particles at the smallest scales, where traditional Newtonian physics no longer applies and instead the principles of uncertainty, superposition, and entanglement govern their actions.


In [ ]:
# help(ChatOpenAI)

In [ ]:
from langchain.schema import SystemMessage, AIMessage, HumanMessage

messages = [
    SystemMessage(content='You are a physicist and respond only in German.'),
    HumanMessage(content='Explain quatum mechanics in one sentence.')
]

output = llm.invoke(messages)
print(output.content)

Quantenmechanik ist ein physikalisches Theoriegebäude, das das Verhalten von Teilchen auf atomarer und subatomarer Ebene beschreibt.


# Caching LLM Responses

## In-Memory Cache

In [11]:
pip install langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 1.8 MB/s eta 0:00:00


In [ ]:
from langchain.globals import set_llm_cache
from langchain_openai import OpenAI
llm = OpenAI(model_name='gpt-3.5-turbo-instruct') # To make the caching really obvious, lets use a slower and older model.

In [ ]:
%%time
from langchain.cache import InMemoryCache
set_llm_cache(InMemoryCache())
prompt = 'Tell me a joke that a toddler can understand.'
llm.invoke(prompt)

CPU times: user 1.07 s, sys: 56.1 ms, total: 1.13 s
Wall time: 2.19 s


'\nWhy was the broom late for school? Because it overswept!'

In [ ]:
%%time
llm.invoke(prompt)

CPU times: user 1.12 ms, sys: 1e+03 ns, total: 1.12 ms
Wall time: 5.09 ms


'\nWhy was the broom late for school? Because it overswept!'

## SQLite Cache

In [ ]:
from langchain.cache import SQLiteCache
set_llm_cache(SQLiteCache(database_path='langchaind.db'))

# First request (not in cache, takes longer)
llm.invoke('Tell me a joke')

# Second request (cached, faster)
llm.invoke('Tell me a joke')

# LLM Streaming

In [ ]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI()
prompt = 'Write a rock song about the Moon and a Raven.'
print(llm.invoke(prompt).content)

Verse 1:
In the still of the night,
Underneath the moon's soft light,
I hear the raven's cry,
Telling stories of the sky.

Chorus:
Oh, Moon and Raven,
Dancing in the night,
Mystical and enchanting,
In their own moonlit delight.

Verse 2:
The raven flies high,
As the moon watches from the sky,
Their spirits intertwined,
In this cosmic design.

Chorus:
Oh, Moon and Raven,
Dancing in the night,
Mystical and enchanting,
In their own moonlit delight.

Bridge:
The moon whispers secrets,
As the raven sings its song,
Together they create a symphony,
That will last all night long.

Chorus:
Oh, Moon and Raven,
Dancing in the night,
Mystical and enchanting,
In their own moonlit delight.

Outro:
As the night fades away,
The moon and raven will stay,
Forever connected in the night,
In a never-ending flight.


In [ ]:
for chunk in llm.stream(prompt):
  print(chunk.content, end='', flush=True)

Verse 1:
In the dead of night, the moon appears so bright
Casting shadows on the land below
And in the darkness, a raven takes flight
Singing a song only he knows

Chorus:
Oh moon, shining bright
Guide the raven through the night
Oh raven, take flight
Sing your song under the moonlight

Verse 2:
The raven soars high, against the midnight sky
Whispering secrets to the moon
As the world sleeps, they keep their watchful eye
In this cosmic dance, they're in tune

Chorus:
Oh moon, shining bright
Guide the raven through the night
Oh raven, take flight
Sing your song under the moonlight

Bridge:
They're two creatures of the night
Bound by a mysterious light
A dance of shadows in the dark
Their connection leaves its mark

Chorus:
Oh moon, shining bright
Guide the raven through the night
Oh raven, take flight
Sing your song under the moonlight

Outro:
In the stillness of the night, they roam
The moon and raven, forever entwined
A haunting melody in the silent gloam
Their spirits intertwined.

# PromptTemplates


In [ ]:
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

template = '''You are an experience virologist.
Write a few sentences about the following virus "{virus}" in {language}.'''

prompt_template = PromptTemplate(template=template)

prompt = prompt_template.format(virus='hiv', language='german')
prompt

'You are an experience virologist.\nWrite a few sentences about the following virus "hiv" in german.'

In [ ]:
llm = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0)
output = llm.invoke(prompt).content
print(output)

HIV, das humane Immundefizienzvirus, ist ein Virus, das das Immunsystem des Menschen schwächt. Es wird hauptsächlich durch ungeschützten Geschlechtsverkehr, den Austausch von infizierten Nadeln oder von der Mutter auf das Kind während der Schwangerschaft übertragen. HIV kann zu AIDS führen, einer schweren Erkrankung, die das Immunsystem so schwächt, dass der Körper anfällig für verschiedene Infektionen und Krankheiten wird. Es ist wichtig, sich über die Übertragungswege von HIV zu informieren und sich vor einer Infektion zu schützen.


# ChatPromptTemplates

In [ ]:
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate
from langchain_core.messages import SystemMessage

chat_template = ChatPromptTemplate(
    [
        SystemMessage(content='You respond only in the JSON format.'),
        HumanMessagePromptTemplate.from_template('Top {n} countries in {area} by population.')
    ]
)

messages = chat_template.format_messages(n='10', area='World')
print(messages)

[SystemMessage(content='You respond only in the JSON format.', additional_kwargs={}, response_metadata={}), HumanMessage(content='Top 10 countries in World by population.', additional_kwargs={}, response_metadata={})]


In [ ]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI()
output = llm.invoke(messages).content
print(output)

{
    "1": {
        "country": "China",
        "population": "1,415,045,928"
    },
    "2": {
        "country": "India",
        "population": "1,354,051,854"
    },
    "3": {
        "country": "United States",
        "population": "326,766,748"
    },
    "4": {
        "country": "Indonesia",
        "population": "266,794,980"
    },
    "5": {
        "country": "Pakistan",
        "population": "200,813,818"
    },
    "6": {
        "country": "Brazil",
        "population": "190,732,694"
    },
    "7": {
        "country": "Nigeria",
        "population": "185,989,640"
    },
    "8": {
        "country": "Bangladesh",
        "population": "157,826,578"
    },
    "9": {
        "country": "Russia",
        "population": "142,257,519"
    },
    "10": {
        "country": "Mexico",
        "population": "129,163,276"
    }
}


# Simple Chains

In [ ]:
from langchain_openai import ChatOpenAI
from langchain import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

llm = ChatOpenAI()
template = '''You are an experience virologist.
Write a few sentences about the following virus "{virus}" in {language}.'''
prompt_template = PromptTemplate.from_template(template=template)

chain = prompt_template | llm | StrOutputParser()

output = chain.invoke({'virus': 'HSV', 'language': 'Spanish'})

In [ ]:
print(output)

El virus del herpes simple (HSV) es un virus altamente contagioso que afecta principalmente a la piel y las membranas mucosas. Hay dos tipos de HSV: el tipo 1, que suele causar herpes labial en la boca y alrededor de los labios, y el tipo 2, que suele causar herpes genital. Ambos tipos de HSV son crónicos y pueden producir brotes recurrentes a lo largo de la vida de una persona infectada. Es importante practicar una buena higiene y evitar el contacto directo con lesiones activas para prevenir la transmisión del virus.


In [ ]:
template = 'What is the capital of {country}?. List the top 3 places to visit in that city. Use bullet points.'
prompt_template = PromptTemplate.from_template(template=template)

chain = prompt_template | llm | StrOutputParser()

country = input('Enter Country: ')
output = chain.invoke(country)
print(output)

Enter Country: Brazil
The capital of Brazil is Brasília.

- Juscelino Kubitschek Memorial: This museum pays tribute to Juscelino Kubitschek, the president who established Brasília as the country's capital.
- National Congress: This iconic building houses both the Chamber of Deputies and the Federal Senate of Brazil, and is a must-visit for anyone interested in politics.
- Planalto Palace: This is the official workplace of the President of Brazil and is an important landmark in Brasília.


# Sequential Chains

In [ ]:
from langchain_openai import ChatOpenAI
from langchain import PromptTemplate
# from langchain.chains import SimpleSequentialChain
from langchain_core.output_parsers import StrOutputParser

llm1 = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0.5)
prompt_template1 = PromptTemplate.from_template(
    template='You are an experienced scientist and Python programmer. Write a function that implements the concept of {concept}.'
)
chain1 = prompt_template1 | llm1 | StrOutputParser()

llm2 = ChatOpenAI(model_name='gpt-4-turbo-preview', temperature=1.2)
prompt_template2 = PromptTemplate.from_template(
    template = 'Given the Python function {function}, describe it as detailed as possible.'
)
chain2 = prompt_template2 | llm2 | StrOutputParser()

overall_chain = chain1 | chain2
output = overall_chain.invoke('linear regression')

In [ ]:
print(output)

The Python function `linear_regression` is designed to perform linear regression, a fundamental method in statistical modeling and machine learning. Linear regression aims to model the linear relationship between a dependent variable and one or more independent variables by fitting a linear equation to observed data. The goal is to find the coefficients (or parameters) of the linear equation that best predict the dependent variable. Here, the function focuses on simple linear regression, where there is just one independent variable, though the method can be slightly adjusted to handle multiple variables. Below is a detailed description of the function’s implementation and its contents.

### Importing Necessary Libraries

The function begins by importing NumPy, a widely used library in Python for numerical computing. NumPy provides efficient data structures for arrays and matrices, along with a collection of mathematical functions to operate on these data structures, which is essential 

# LangChain Agents in Action: Python REPL

In [12]:
pip install -q langchain_experimental

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.2/209.2 kB 5.3 MB/s eta 0:00:00


In [ ]:
from langchain_experimental.utilities import PythonREPL
python_repl = PythonREPL()
python_repl.run('print([n for n in range(1, 100) if n % 13 == 0])')

'[13, 26, 39, 52, 65, 78, 91]\n'

In [ ]:
from langchain_experimental.agents.agent_toolkits import create_python_agent
from langchain_experimental.tools.python.tool import PythonREPLTool
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model_name='gpt-4-turbo-preview', temperature=0)
agent_executor = create_python_agent(
    llm=llm,
    tool=PythonREPLTool(),
    verbose=True
)

agent_executor.invoke('Calculate the square root of the factorial of 12 and display it with 4 decimal points')



> Entering new AgentExecutor chain...
To solve this, I need to calculate the factorial of 12 first, then find its square root, and finally format the result to display it with 4 decimal points. I can use the `math` module in Python for both the factorial and square root calculations.
Action: Python_REPL
Action Input: import math
print(f"{math.sqrt(math.factorial(12)):.4f}")
Observation: 21886.1052

Thought:I now know the final answer
Final Answer: 21886.1052

> Finished chain.


{'input': 'Calculate the square root of the factorial of 12 and display it with 4 decimal points',
 'output': '21886.1052'}

In [ ]:
response = agent_executor.invoke('What is the answer to 5.1 ** 7.3?')



> Entering new AgentExecutor chain...
I need to calculate 5.1 raised to the power of 7.3 to get the answer.
Action: Python_REPL
Action Input: print(5.1 ** 7.3)
Observation: 146306.05007233328

Thought:I now know the final answer
Final Answer: 146306.05007233328

> Finished chain.


In [ ]:
response

{'input': 'What is the answer to 5.1 ** 7.3?', 'output': '146306.05007233328'}

In [ ]:
print(response['output'])

146306.05007233328


# LangChain Tools: DuckDuckGo and Wikipedia

In [ ]:
pip install -q duckduckgo-search

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 26.4 MB/s eta 0:00:00


In [ ]:
from langchain.tools import DuckDuckGoSearchRun

search = DuckDuckGoSearchRun()
output = search.invoke('Where was Freddie Mercury born?')
print(output)

Freddie Mercury (born September 5, 1946, Stone Town, Zanzibar [now in Tanzania]—died November 24, 1991, Kensington, London, England) was a British rock singer and songwriter whose flamboyant showmanship and powerfully agile vocals, most famously for the band Queen, made him one of rock's most dynamic front men.. Bulsara was born to Parsi parents who had emigrated from India to Zanzibar ... Freddie Mercury, born Farrokh Bulsara on September 5, 1946, in Zanzibar, was a legendary singer-songwriter and the charismatic frontman of the rock band Queen. With a unique background as a Parsi in Tanzania and a music education in India, Mercury made a significant impact on the music world. Freddie Mercury was born Farrokh Bulsara in Stone Town in the British protectorate of the Sultanate of Zanzibar, East Africa (now part of Tanzania) on September 5, 1946. When did he get his start in music? Freddie Mercury began his musical career in the late 1960s. He formed his first band, The Hectics, while at

In [ ]:
search.name

'duckduckgo_search'

In [ ]:
search.description

'A wrapper around DuckDuckGo Search. Useful for when you need to answer questions about current events. Input should be a search query.'

In [ ]:
search

DuckDuckGoSearchRun(api_wrapper=DuckDuckGoSearchAPIWrapper(region='wt-wt', safesearch='moderate', time='y', max_results=5, backend='auto', source='text'))

In [ ]:
from langchain.tools import DuckDuckGoSearchResults

search = DuckDuckGoSearchResults()
output = search.run('Freddie Mercury and Queen')
print(output)

snippet: Freddie Mercury (born September 5, 1946, Stone Town, Zanzibar [now in Tanzania]—died November 24, 1991, Kensington, London, England) was a British rock singer and songwriter whose flamboyant showmanship and powerfully agile vocals, most famously for the band Queen, made him one of rock's most dynamic front men.. Bulsara was born to Parsi parents who had emigrated from India to Zanzibar ..., title: Freddie Mercury | Biography, Parents, Songs, & Facts | Britannica, link: https://www.britannica.com/biography/Freddie-Mercury, snippet: When Freddie Mercury was looking to break free of Queen as his band went on hiatus in 1983, he turned to a king — King of Pop, Michael Jackson. The two late music legends were mutual fans ..., title: Exclusive | Why Freddie Mercury never finished 3 duets with Michael ..., link: https://nypost.com/2025/04/28/entertainment/why-freddie-mercury-never-finished-3-duets-with-michael-jackson-it-involves-a-llama/, snippet: Freddie Mercury of Queen, interview 

In [ ]:
from langchain_community.utilities import DuckDuckGoSearchAPIWrapper

wrapper = DuckDuckGoSearchAPIWrapper(region='de-de', max_results=3, safesearch='moderate')
search = DuckDuckGoSearchResults(api_wrapper=wrapper, source='news')
output = search.run('Berlin')
print(output)

snippet: Erfahren Sie mehr über die Geschichte, Geographie, Politik und Kultur der bevölkerungsreichsten Stadt Deutschlands und der Europäischen Union. Lesen Sie die wichtigsten Fakten, Daten und Sehenswürdigkeiten Berlins auf der umfassenden Online-Enzyklopädie., title: Berlin - Wikipedia, link: https://de.wikipedia.org/wiki/Berlin, snippet: Der Stadtführer für Berlin mit aktuellen Informationen und Auskunft zu Jobs, zum Leben, Arbeiten, Ausgehen, Einkaufen und Urlauben in Berlin., title: Stadtportal Berlin | Stadtinformationen für Berlin bei ... - meinestadt.de, link: https://home.meinestadt.de/berlin, snippet: Einfach zurechtfinden: Mit unserer übersichtlichen Liste der Sehenswürdigkeiten in Berlin planst du deinen Städtetrip ohne Umwege., title: Berlin Sehenswürdigkeiten: Top-25-Liste mit Tipps & Bezirken, link: https://www.voucherwonderland.com/reisemagazin/top-20-sehenswuerdigkeiten-in-berlin/, snippet: Du fragst dich, was man in Berlin machen kann? Wir zeigen dir die besten Tage

In [ ]:
import re
pattern = r'snippet: (.*?), title: (.*?), link: (.*?)(?:,|$)'
matches = re.findall(pattern, output, re.DOTALL)

for snippet, title, link in matches:
  print(f'Snippet: {snippet}\nTitle: {title}\nLink: {link}\n')
  print('-' * 50)

Snippet: Erfahren Sie mehr über die Geschichte, Geographie, Politik und Kultur der bevölkerungsreichsten Stadt Deutschlands und der Europäischen Union. Lesen Sie die wichtigsten Fakten, Daten und Sehenswürdigkeiten Berlins auf der umfassenden Online-Enzyklopädie.
Title: Berlin - Wikipedia
Link: https://de.wikipedia.org/wiki/Berlin

--------------------------------------------------
Snippet: Der Stadtführer für Berlin mit aktuellen Informationen und Auskunft zu Jobs, zum Leben, Arbeiten, Ausgehen, Einkaufen und Urlauben in Berlin.
Title: Stadtportal Berlin | Stadtinformationen für Berlin bei ... - meinestadt.de
Link: https://home.meinestadt.de/berlin

--------------------------------------------------
Snippet: Einfach zurechtfinden: Mit unserer übersichtlichen Liste der Sehenswürdigkeiten in Berlin planst du deinen Städtetrip ohne Umwege.
Title: Berlin Sehenswürdigkeiten: Top-25-Liste mit Tipps & Bezirken
Link: https://www.voucherwonderland.com/reisemagazin/top-20-sehenswuerdigkeiten-in

In [ ]:
pip install -q wikipedia

  Preparing metadata (setup.py) ... done


In [ ]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

api_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=5000)
wiki = WikipediaQueryRun(api_wrapper=api_wrapper)
wiki.invoke({'query': 'vector database'})

'Page: Vector database\nSummary: A vector database, vector store or vector search engine is a database that can store vectors (fixed-length lists of numbers) along with other data items. Vector databases typically implement one or more Approximate Nearest Neighbor algorithms, so that one can search the database with a query vector to retrieve the closest matching database records.\nVectors are mathematical representations of data in a high-dimensional space. In this space, each dimension corresponds to a feature of the data, with the number of dimensions ranging from a few hundred to tens of thousands, depending on the complexity of the data being represented. A vector\'s position in this space represents its characteristics. Words, phrases, or entire documents, as well as images, audio, and other types of data, can all be vectorized.\nThese feature vectors may be computed from the raw data using machine learning methods such as feature extraction algorithms, word embeddings or deep le

In [ ]:
wiki.invoke('Google Gemini')

'Page: Gemini (chatbot)\nSummary: Gemini, formerly known as Bard, is a generative artificial intelligence chatbot developed by Google. Based on the large language model (LLM) of the same name, it was launched in 2023 in response to the rise of OpenAI\'s ChatGPT. It was previously based on the LaMDA and PaLM LLMs.\nLaMDA had been developed and announced in 2021, but it was not released to the public out of an abundance of caution. OpenAI\'s launch of ChatGPT in November 2022 and its subsequent popularity caught Google executives off-guard, prompting a sweeping response in the ensuing months. After mobilizing its workforce, the company launched Bard in a limited capacity in March 2023 before expanding to other countries in May. Bard took center stage during the 2023 Google I/O keynote in May and was upgraded to the Gemini LLM in December. In February 2024, Bard and Duet AI, another artificial intelligence product from Google, were unified under the Gemini brand, coinciding with the launc

# Creating a React Agent

In [19]:
pip install langchainhub -q

SyntaxError: invalid syntax (<ipython-input-19-82c9e354a355>, line 1)

In [9]:
import os
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)

# os.environ.get('OPENAI_API_KEY')

True

In [22]:
pip install -q duckduckgo-search

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 51.0 MB/s eta 0:00:00


In [23]:
pip install wikipedia -q

  Preparing metadata (setup.py) ... done


In [37]:
from langchain.prompts import PromptTemplate
from langchain import hub
from langchain.agents import Tool, AgentExecutor, initialize_agent, create_react_agent
from langchain.tools import DuckDuckGoSearchRun, WikipediaQueryRun
from langchain.utilities import WikipediaAPIWrapper
from langchain_experimental.tools.python.tool import PythonREPLTool
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model_name='gpt-4-turbo-preview', temperature=0) # gpt-3.5-turbo for free access

# Template
template = '''
Answer the following questions in GERMAN as best you can.
Questions: {q}
'''

prompt_template = PromptTemplate.from_template(template)
prompt = hub.pull('hwchase17/react')
# print(type(prompt))
# print(prompt.input_variables)
# print(prompt.template)

# 1. Python REPL Tool (for executing Python code)
python_repl = PythonREPLTool()
python_repl_tool = Tool(
    name='Python REPL',
    func=python_repl.run,
    description='Useful when you need to use Python to answer a question. You should input Python code.'
)

# 2. Wikipedia Tool (for searching Wikipedia)
api_wrapper = WikipediaAPIWrapper()
wikipedia = WikipediaQueryRun(api_wrapper=api_wrapper)
wikipedia_tool = Tool(
    name='Wikipedia',
    func=wikipedia.run,
    description='Useful for when you need to look up a topic, country, or person on Wikipedia.'
)

# 3. DuckDuckGo Search Tool (for general web searches)
search = DuckDuckGoSearchRun()
duckduckgo_tool = Tool(
    name='DuckDuckGo Search',
    func=search.run,
    description='Useful when you need to perform an internet search to find information that another tool can\'t provide.'
)

# Tools
tools = [python_repl_tool, wikipedia_tool, duckduckgo_tool]

# Agent
agent = create_react_agent(llm, tools, prompt)
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
    handle_parsing_errors=True,
    max_iterations=10
)

/usr/local/lib/python3.11/dist-packages/langsmith/client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [38]:
# question = 'Generate the first 20 numbers in the Fibonacci series.'
# question = 'Who is the current prime minister of the UK?'
question = 'Tell me about Napoleon Bonaparte early life'

output = agent_executor.invoke({
    'input': prompt_template.format(q=question)
})



> Entering new AgentExecutor chain...
To answer this question in German, I will first find information about Napoleon Bonaparte's early life in English and then translate it into German.

Action: Wikipedia
Action Input: Napoleon BonapartePage: Napoleon
Summary: Napoleon Bonaparte (born Napoleone Buonaparte; 15 August 1769 – 5 May 1821), later known by his regnal name Napoleon I, was a French general and statesman who rose to prominence during the French Revolution and led a series of military campaigns across Europe during the French Revolutionary and Napoleonic Wars from 1796 to 1815. He led the French Republic as First Consul from 1799 to 1804, then ruled the French Empire as Emperor of the French from 1804 to 1814, and briefly again in 1815. He was King of Italy from 1805 to 1814 and Protector of the Confederation of the Rhine from 1806 to 1813.
Born on the island of Corsica to a family of Italian origin, Napoleon moved to mainland France in 1779 and was commissioned as an officer

In [39]:
for i in output.keys():
  print(i, output[i])

input 
Answer the following questions in GERMAN as best you can.
Questions: Tell me about Napoleon Bonaparte early life

output Napoleon Bonaparte wurde am 15. August 1769 auf der Insel Korsika in eine Familie italienischer Herkunft geboren. Im Jahr 1779 zog Napoleon nach Frankreich und wurde 1785 als Offizier in die französische Königsarmee aufgenommen. Er unterstützte die Französische Revolution von 1789 und förderte ihre Sache auf Korsika. Durch den Sieg bei der Belagerung von Toulon im Jahr 1793 und die Niederlage der royalistischen Aufständischen in Paris am 13. Vendémiaire im Jahr 1795 stieg er schnell in den militärischen Rängen auf. Im Jahr 1796 befehligte Napoleon eine militärische Kampagne gegen die Österreicher und ihre italienischen Verbündeten im Krieg der Ersten Koalition, erzielte entscheidende Siege und wurde zum Nationalhelden.


## Adding a new tool

In [40]:
pip install arxiv -q

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 3.8 MB/s eta 0:00:00


In [41]:
from langchain.prompts import PromptTemplate
from langchain import hub
from langchain.agents import Tool, AgentExecutor, initialize_agent, create_react_agent
from langchain.tools import DuckDuckGoSearchRun, WikipediaQueryRun
from langchain.utilities import WikipediaAPIWrapper, ArxivAPIWrapper
from langchain_experimental.tools.python.tool import PythonREPLTool
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model_name='gpt-4-turbo-preview', temperature=0) # gpt-3.5-turbo for free access

# Template
template = '''
Answer the following questions in GERMAN as best you can.
Questions: {q}
'''

prompt_template = PromptTemplate.from_template(template)
prompt = hub.pull('hwchase17/react')
# print(type(prompt))
# print(prompt.input_variables)
# print(prompt.template)

# 1. Python REPL Tool (for executing Python code)
python_repl = PythonREPLTool()
python_repl_tool = Tool(
    name='Python REPL',
    func=python_repl.run,
    description='Useful when you need to use Python to answer a question. You should input Python code.'
)

# 2. Wikipedia Tool (for searching Wikipedia)
api_wrapper = WikipediaAPIWrapper()
wikipedia = WikipediaQueryRun(api_wrapper=api_wrapper)
wikipedia_tool = Tool(
    name='Wikipedia',
    func=wikipedia.run,
    description='Useful for when you need to look up a topic, country, or person on Wikipedia.'
)

# 3. DuckDuckGo Search Tool (for general web searches)
search = DuckDuckGoSearchRun()
duckduckgo_tool = Tool(
    name='DuckDuckGo Search',
    func=search.run,
    description='Useful when you need to perform an internet search to find information that another tool can\'t provide.'
)

# 4. Arxiv Search Tool (for scientific papers)
arxiv = ArxivAPIWrapper()
arxiv_tool = Tool(
    name='Arxiv Search',
    func = arxiv.run,
    description='Useful for finding scientific papers and abstracts from arXiv. Input should be a research topic.'
)

# Tools
tools = [python_repl_tool, wikipedia_tool, duckduckgo_tool, arxiv_tool]

# Agent
agent = create_react_agent(llm, tools, prompt)
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
    handle_parsing_errors=True,
    max_iterations=10
)

/usr/local/lib/python3.11/dist-packages/langsmith/client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [44]:
# question = 'Generate the first 20 numbers in the Fibonacci series.'
# question = 'Who is the current prime minister of the UK?'
# question = 'Tell me about Napoleon Bonaparte early life'
question = 'Find the most recient papers about Deep Learning. The output has to be a list'

output = agent_executor.invoke({
    'input': prompt_template.format(q=question)
})



> Entering new AgentExecutor chain...
To find the most recent papers about Deep Learning, I should use the Arxiv Search tool since it specializes in finding scientific papers and abstracts.

Action: Arxiv Search
Action Input: Deep LearningPublished: 2018-05-22
Title: Opening the black box of deep learning
Authors: Dian Lei, Xiaoxiao Chen, Jianfei Zhao
Summary: The great success of deep learning shows that its technology contains
profound truth, and understanding its internal mechanism not only has important
implications for the development of its technology and effective application in
various fields, but also provides meaningful insights into the understanding of
human brain mechanism. At present, most of the theoretical research on deep
learning is based on mathematics. This dissertation proposes that the neural
network of deep learning is a physical system, examines deep learning from
three different perspectives: microscopic, macroscopic, and physical world
views, answers multipl